# A - Using TorchText with Your Own Datasets

In this series we have used the IMDb dataset included as a dataset in TorchText. TorchText has many canonical datasets included for classification, language modelling, sequence tagging, etc. However, frequently you'll be wanting to use your own datasets. Luckily, TorchText has functions to help you to this.

Recall in the series, we:
- defined the `Field`s
- loaded the dataset
- created the splits

As a reminder, the code is shown below:

```python
TEXT = data.Field()
LABEL = data.LabelField()

train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)

train_data, valid_data = train_data.split()
```

There are three data formats TorchText can read: `json`, `tsv` (tab separated values) and`csv` (comma separated values).

**In my opinion, the best formatting for TorchText is `json`, which I'll explain later on.**

## Reading JSON

Starting with `json`, your data must be in the `json lines` format, i.e. it must be something like:

```
{"name": "John", "location": "United Kingdom", "age": 42, "quote": ["i", "love", "the", "united kingdom"]}
{"name": "Mary", "location": "United States", "age": 36, "quote": ["i", "want", "more", "telescopes"]}
```

That is, each line is a `json` object. See `data/train.json` for an example.

We then define the fields:

In [1]:
from torchtext import data
from torchtext import datasets

NAME = data.Field()
SAYING = data.Field()
PLACE = data.Field()

Next, we must tell TorchText which fields apply to which elements of the `json` object. 

For `json` data, we must create a dictionary where:
- the key matches the key of the `json` object
- the value is a tuple where:
  - the first element becomes the batch object's attribute name
  - the second element is the name of the `Field`
  
What do we mean when we say "becomes the batch object's attribute name"? Recall in the previous exercises where we accessed the `TEXT` and `LABEL` fields in the train/evaluation loop by using `batch.text` and `batch.label`, this is because TorchText sets the batch object to have a `text` and `label` attribute, each being a tensor containing either the text or the label.

A few notes:

* The order of the keys in the `fields` dictionary does not matter, as long as its keys match the `json` data keys.

- The `Field` name does not have to match the key in the `json` object, e.g. we use `PLACE` for the `"location"` field.

- When dealing with `json` data, not all of the keys have to be used, e.g. we did not use the `"age"` field.

- Also, if the values of `json` field are a string then the `Fields` tokenization is applied (default is to split the string on spaces), however if the values are a list then no tokenization is applied. Usually it is a good idea for the data to already be tokenized into a list, this saves time as you don't have to wait for TorchText to do it.

- The value of the `json` fields do not have to be the same type. Some examples can have their `"quote"` as a string, and some as a list. The tokenization will only get applied to the ones with their `"quote"` as a string.

- If you are using a `json` field, every single example must have an instance of that field, e.g. in this example all examples must have a name, location and quote. However, as we are not using the age field, it does not matter if an example does not have it.

In [2]:
fields = {'name': ('n', NAME), 'location': ('p', PLACE), 'quote': ('s', SAYING)}

Now, in a training loop we can iterate over the data iterator and access the name via `batch.n`, the location via `batch.p`, and the quote via `batch.s`.

We then create our datasets (`train_data` and `test_data`) with the `TabularDataset.splits` function. 

The `path` argument specifices the top level folder common among both datasets, and the `train` and `test` arguments specify the filename of each dataset, e.g. here the train dataset is located at `data/train.json`.

We tell the function we are using `json` data, and pass in our `fields` dictionary defined previously.

In [3]:
train_data, test_data = data.TabularDataset.splits(
                            path = 'data',
                            train = 'train.json',
                            test = 'test.json',
                            format = 'json',
                            fields = fields
)

If you already had a validation dataset, the location of this can be passed as the `validation` argument.

In [4]:
train_data, valid_data, test_data = data.TabularDataset.splits(
                                        path = 'data',
                                        train = 'train.json',
                                        validation = 'valid.json',
                                        test = 'test.json',
                                        format = 'json',
                                        fields = fields
)

We can then view an example to make sure it has worked correctly.

Notice how the field names (`n`, `p` and `s`) match up with what was defined in the `fields` dictionary.

Also notice how the word `"United Kingdom"` in `p` has been split by the tokenization, whereas the `"united kingdom"` in `s` has not. This is due to what was mentioned previously, where TorchText assumes that any `json` fields that are lists are already tokenized and no further tokenization is applied. 

In [5]:
print(vars(train_data[0]))

{'n': ['John'], 'p': ['United', 'Kingdom'], 's': ['i', 'love', 'the', 'united kingdom']}


We can now use `train_data`, `test_data` and `valid_data` to build a vocabulary and create iterators, as in the other notebooks. We can access all attributes by using `batch.n`, `batch.p` and `batch.s` for the names, places and sayings, respectively.

## Reading CSV/TSV

`csv` and `tsv` are very similar, except csv has elements separated by commas and tsv by tabs.

Using the same example above, our `tsv` data will be in the form of:

```
name	location	age	quote
John	United Kingdom	42	i love the united kingdom
Mary	United States	36	i want more telescopes
```

That is, on each row the elements are separated by tabs and we have one example per row. The first row is usually a header (i.e. the name of each of the columns), but your data could have no header.

You cannot have lists within `tsv` or `csv` data.

The way the fields are defined is a bit different to `json`. We now use a list of tuples, where each element is also a tuple. The first element of these inner tuples will become the batch object's attribute name, second element is the `Field` name.

Unlike the `json` data, the tuples have to be in the same order that they are within the `tsv` data. Due to this, when skipping a column of data a tuple of `None`s needs to be used, if not then our `SAYING` field will be applied to the `age` column of the `tsv` data and the `quote` column will not be used. 

However, if you only wanted to use the `name` and `age` column, you could just use two tuples as they are the first two columns.

We change our `TabularDataset` to read the correct `.tsv` files, and change the `format` argument to `'tsv'`.

If your data has a header, which ours does, it must be skipped by passing `skip_header = True`. If not, TorchText will think the header is an example. By default, `skip_header` will be `False`.

In [6]:
fields = [('n', NAME), ('p', PLACE), (None, None), ('s', SAYING)]

In [7]:
train_data, valid_data, test_data = data.TabularDataset.splits(
                                        path = 'data',
                                        train = 'train.tsv',
                                        validation = 'valid.tsv',
                                        test = 'test.tsv',
                                        format = 'tsv',
                                        fields = fields,
                                        skip_header = True
)

In [8]:
print(vars(train_data[0]))

{'n': ['John'], 'p': ['United', 'Kingdom'], 's': ['i', 'love', 'the', 'united', 'kingdom']}


Finally, we'll cover `csv` files. 

This is pretty much the exact same as the `tsv` files, expect with the `format` argument set to `'csv'`.

In [9]:
fields = [('n', NAME), ('p', PLACE), (None, None), ('s', SAYING)]

In [10]:
train_data, valid_data, test_data = data.TabularDataset.splits(
                                        path = 'data',
                                        train = 'train.csv',
                                        validation = 'valid.csv',
                                        test = 'test.csv',
                                        format = 'csv',
                                        fields = fields,
                                        skip_header = True
)

In [11]:
print(vars(train_data[0]))

{'n': ['John'], 'p': ['United', 'Kingdom'], 's': ['i', 'love', 'the', 'united', 'kingdom']}


## Why JSON over CSV/TSV?

1. Your `csv` or `tsv` data cannot be stored lists. This means data cannot be already be tokenized, thus everytime you run your Python script that reads this data via TorchText, it has to be tokenized. Using advanced tokenizers, such as the `spaCy` tokenizer, takes a non-negligible amount of time. Thus, it is better to tokenize your datasets and store them in the `json lines` format.

2. If tabs appear in your `tsv` data, or commas appear in your `csv` data, TorchText will think they are delimiters between columns. This will cause your data to be parsed incorrectly. Worst of all TorchText will not alert you to this as it cannot tell the difference between a tab/comma in a field and a tab/comma as a delimiter. As `json` data is essentially a dictionary, you access the data within the fields via its key, so do not have to worry about "surprise" delimiters.

## Iterators 

Using any of the above datasets, we can then build the vocab and create the iterators.

In [12]:
NAME.build_vocab(train_data)
SAYING.build_vocab(train_data)
PLACE.build_vocab(train_data)

Then, we can create the iterators after defining our batch size and device.

By default, the train data is shuffled each epoch, but the validation/test data is sorted. However, TorchText doesn't know what to use to sort our data and it would throw an error if we don't tell it. 

There are two ways to handle this, you can either tell the iterator not to sort the validation/test data by passing `sort = False`, or you can tell it how to sort the data by passing a `sort_key`. A sort key is a function that returns a key on which to sort the data on. For example, `lambda x: x.s` will sort the examples by their `s` attribute, i.e their quote. Ideally, you want to use a sort key as the `BucketIterator` will then be able to sort your examples and then minimize the amount of padding within each batch.

We can then iterate over our iterator to get batches of data. Note how by default TorchText has the batch dimension second.

In [13]:
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

BATCH_SIZE = 1

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data),
    sort = False, #don't sort test/validation data
    batch_size=BATCH_SIZE,
    device=device)

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data),
    sort_key = lambda x: x.s, #sort by s attribute (quote)
    batch_size=BATCH_SIZE,
    device=device)

print('Train:')
for batch in train_iterator:
    print(batch)
    
print('Valid:')
for batch in valid_iterator:
    print(batch)
    
print('Test:')
for batch in test_iterator:
    print(batch)

Train:

[torchtext.data.batch.Batch of size 1]
	[.n]:[torch.cuda.LongTensor of size 1x1 (GPU 0)]
	[.p]:[torch.cuda.LongTensor of size 2x1 (GPU 0)]
	[.s]:[torch.cuda.LongTensor of size 5x1 (GPU 0)]

[torchtext.data.batch.Batch of size 1]
	[.n]:[torch.cuda.LongTensor of size 1x1 (GPU 0)]
	[.p]:[torch.cuda.LongTensor of size 2x1 (GPU 0)]
	[.s]:[torch.cuda.LongTensor of size 4x1 (GPU 0)]
Valid:

[torchtext.data.batch.Batch of size 1]
	[.n]:[torch.cuda.LongTensor of size 1x1 (GPU 0)]
	[.p]:[torch.cuda.LongTensor of size 1x1 (GPU 0)]
	[.s]:[torch.cuda.LongTensor of size 2x1 (GPU 0)]

[torchtext.data.batch.Batch of size 1]
	[.n]:[torch.cuda.LongTensor of size 1x1 (GPU 0)]
	[.p]:[torch.cuda.LongTensor of size 1x1 (GPU 0)]
	[.s]:[torch.cuda.LongTensor of size 4x1 (GPU 0)]
Test:

[torchtext.data.batch.Batch of size 1]
	[.n]:[torch.cuda.LongTensor of size 1x1 (GPU 0)]
	[.p]:[torch.cuda.LongTensor of size 1x1 (GPU 0)]
	[.s]:[torch.cuda.LongTensor of size 3x1 (GPU 0)]

[torchtext.data.batch.Batch o